## Obtención de columnas con persistencia temporal y agrupación de Promedio, máximo y mínimo por hora.

#### Diseño del modelo

Una vez que se obtienen los datos limpios, estos pueden ser utilizados para una serie de pruebas, donde se decidirá la granularidad temporal y la representatividad geográfica con la que se va a efectuar el pronóstico. Este proceso es iterativo: se aplica un procesamiento a los datos, se entrenan los modelos y se analiza su desempeño. A continuación se muestran las principales variantes de los modelos generados así como sus descripción:


<img src="./fotos_markdown/diseno_modelo.png" style="height:300px">

* __Pronóstico por estación__: Se generó un modelo para todas las estaciónes de monitoreo. Debido a que la mayoría de las estaciones solo miden un subconjunto de todas las variables y no todas a la vez, los modelos generados sólo resultaron viables para las estaciones que miden todas las variables. Para la mayoría de las estaciones donde la cantidad de variables es baja,  el modelo subestima los valores de pronóstico.


* __Pronóstico para toda la Ciudad__: Para poder solventar que no todas las estaciones miden las mismas variables, se obtuvo el promedio, el mínimo y el máximo de los valores medidos en todas las estaciones por cada hora, al tomar estos valores, se pierde el atributo de la estación. El pronóstico mejoró, sin embargo el desempeño no era el esperado debido a los numerosos falsos positivos: la información-base de los modelos no era suficiente para describir el fenómeno.


* __Pronóstico por día__: Junto al proceso anterior, se procedió a volver a sacar el promedio, el mínimo y el máximo pero en esta ocasión por día y no por hora. Se obtuvo un modelo de pronóstico del valor máximo de cada día. El resultado no fue el esperado, el resultado mejoró considerablemente a comparación de los dos procesos anteriores sin embargo, el modelo seguía pronosticando demasiados falsos negativos: la contingencia no se pronosticaba correctamente debido a que al promediar por día, se pierde la información horaria y disminuye considerablemente el número de registros.


* __Pronóstico con persistencia temporal__: Usando los datos del pronóstico por hora para toda la ciudad, se utilizó la persistencia observada en el fenómeno entre horas pasadas y presentes, para generar nuevas columnas con los atributos de horas pasadas más correlacionadas con el presente. Se volvieron a entrenar los modelos con estas variables nuevas, obteniendo resultados satisfactorios.


__Pasos y descripción general del notebook__


El modelo elegido es el __horario con persistencia temporal__, habiendo obtenido la más baja tasa de falsos negativos y falsos positivos. Más adelante se expone su análisis con métricas de desempeño. A continuación se muestra el proceso para obtener los datos de entrenamiento del modelo:

<img src="./fotos_markdown/diagrama_2.png" style="height:350px">



1. __Contaminantes de horas anteriores más correlacionadas:__ se obtiene la correlación, de las variables meteorológicas y de contaminación, entre sus valores de una hora atrás con los de 72 horas atrás consigo mismas, de 1 hasta 72 horas atrás. El resultado es una una tabla de correlación temporal desplazada temporalmente la cual nos servirá para poder filtrar las variables más correlacionadas, es decir, las que más influyen en los niveles de contaminación.


2. __Aplicar Desplazamiento temporal:__ a partir de tablas de correlación temporal desplazada, se hace una columna para cada variable correlacionada: cuando existe una correlación mayor a 0.44 de cualquier atributo con los contaminantes mencionados, se hace una nueva columna con estos valores.



3. __Max, Mean, Min:__ se obtiene el máximo, el promedio y el mínimo de cada variable por hora, el resultado es una tabla con una sola fila por cada hora, en esta tabla se pierde el atributo de estación y así la información geográfica de donde se tomó la medición.


4. __Generar las columna a pronosticar:__ Se genera la columna con el valor del conntaminante a pronosticar a 24 horas y se obtiene la correlación de los valores promedio máximo y mínimo con éste. Por último se Filtran los valores más importantes.


5. __Generar las columnas de pronóstico de RH, WSP TMP:__ Se generan columnas con el pronóstico meteorológico  de estos atributos.


6. __Generar las columnas del contaminante a pronosticar de 1 hasta 23 horas:__ Se generan las columnas con el valor del contaminante a prononsticar desde 1 hasta 24 horas.



- __Datos recibidos:__ Limpieza de datos inicial
- __Responsable:__ Daniel Bustillos
- __Contacto:__  juandaniel.bucam@gmail.com

Este notebook presenta el proceso donde se ha obtenido los mejores desempeños de los modelos de los contaminantes. Después de un proceso iterativo y de un análisis del desempeño de los modelos, se obtuvieron las siguientes conclusiones:

- Utilizar la propiedad de persistencia temporal, mejora significativamente los modelos.


- Obtener el promedio, el máximo y el mínimo por hora en lugar de trabajar con la medición de cada estación, genera una consolidación de las mediciones y disminuye el efecto negativo de los atributos faltantes en las mediciones a raíz de las capacidades de las estación de monitoreo.


- Añadir el pronóstico meteorológico a 24 horas, añade fiabilidad a los modelos disminuyendo el error asociado del pronóstico. Este pronóstico se obtendrá posteriormente de una fuente externa.


In [1]:
Contaminantes = ["O3","PM10mean","PM25mean"]

In [29]:
contaminante = Contaminantes[0]
contaminante

'O3'

In [30]:
import pandas as pd
pd.options.mode.chained_assignment = None 

In [31]:
data_hour_merge_24 = pd.read_csv('./datos/datos_entrenamiento.csv')

In [32]:
data_hour_merge_24 = data_hour_merge_24.dropna(subset=[contaminante]).reset_index(drop=True)#PM25

In [33]:
data_hour_merge_24.head(5)

,CO,NO,NO2,NOX,O3,PM10,PM2.5,RH,SO2,TMP,WSP,dia,fecha,hora,id_station,mes,PM10mean,PM25mean
0,0.6,14.0,21.0,35.0,4.0,84.0,NaN,82.0,4.0,9.0,1.8,1.0,2015-01-01 01:00:00,1,ACO,1.0,NaN,NaN
1,1.2,3.0,58.0,61.0,8.0,NaN,NaN,NaN,20.0,NaN,1.3,1.0,2015-01-01 01:00:00,1,AJM,1.0,NaN,NaN
2,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2015-01-01 01:00:00,1,AJU,1.0,NaN,NaN
3,1.1,3.0,30.0,32.0,21.0,122.0,NaN,NaN,16.0,NaN,NaN,1.0,2015-01-01 01:00:00,1,ATI,1.0,NaN,NaN
4,1.2,NaN,NaN,NaN,2.0,95.0,80.0,NaN,67.0,NaN,NaN,1.0,2015-01-01 01:00:00,1,CAM,1.0,NaN,NaN


## 1. 
### Usando el archivo de correlación con retraso temporal, se crean nuevas columnas con el atributo y las horas atrás.


La corraloción con retraso temporal se refiere a la influencia que tienen las condiciones atmosféricas de un hora dada sobre las condiciones futuras, por ejemplo, si el nivel de contaminación de actual es alto ¿este valor influirá en la contaminación del día siguiente o la contaminación de cada día no se ve influida por valores pasados? La respuesta es la primera opción, valores pasados de las condiciones atmosféricas influyen fuertemente en la contaminación _actual_.

Esta propiedad es clave para la generación de un modelo estadístico, la existencia de patrones y de factores que aparentemente influyen en la contaminación nos permiten tener un conjunto de datos que determinen la contaminación.


La generación de la correlación se encuentra en: __/home/paw/DanielBustillos/contaminación/correlaciones_pau/correlaciones_función_paulina.ipynb__

In [34]:
target = str(contaminante + "_y")
target

'O3_y'

In [35]:
O3_corr = pd.read_csv("./correlacion/correlaciones_"+ target +".csv") # leemo el archivo de correlación
O3_corr = O3_corr.iloc[:,1:]
O3_corr.tail()

,contaminante,valor,horas
1067,PM25mean,0.165820,44.0
1068,PM25mean,0.167111,45.0
1069,PM25mean,0.168421,46.0
1070,PM25mean,0.169548,47.0
1071,PM25mean,0.170081,48.0


- La columna "contaminante" se tiene el atributo sobre el que se realizó la correlación.
- La columna "valor" indica el valor de la correlación.
- La columna "horas" indica las horas de retraso.

Vamos a aplicar un filtro, nos quedaremos con las variables con una correlación entre -0.44 y 0.44. Este valor se eligió de forma que la cantidad de atributos final sea menor a 110:

In [36]:
if contaminante == "O3":
    threshold = 0.4
else: threshold = 0.5

In [37]:
O3_corr_filtro = O3_corr[(O3_corr.valor > threshold ) | (O3_corr.valor < - threshold) ].reset_index(drop=True)
O3_corr_filtro.shape

(147, 3)

Una vez que tenemos los contaminantes y el número de horas de retraso, definimos una función que nos genere una nueva columna con el contaminante recorrido temporalmente las horas indicadas en **O3_corr_filtro**:

In [38]:
def shit_corr(df):
    for i in range(len(O3_corr_filtro)):
        name_column = str( O3_corr_filtro.loc[i,"contaminante"] + "_" + str(
            O3_corr_filtro.loc[i,"horas"])) # se define el nombre de la columna a crear con el formato:
                                            # contaminante _ horas de retraso
        df[name_column] = df[O3_corr_filtro.loc[i,"contaminante"]].shift(
            int(float(str(O3_corr_filtro.loc[i,"horas"])))) # se toma el nombre del contaminante y se 
                                                            # desplaza temporalmente las horas indicadas
    return df.dropna()

Debido a que por cada hora se tienen tantos registros como el número de estaciones de monitoreo, existen fechas duplicadas en el df:

Al tener fechas duplicadas, puede ocurrrir errores al aplicar el desplazamiento, por este motivo vamos a generar **un dataframe por cada estación**, aplicaremos la función y por último uniremos los dataframes:

In [39]:
[str( O3_corr_filtro.loc[i,"contaminante"] + "_" + str(O3_corr_filtro.loc[i,"horas"])) for i in range(len(O3_corr_filtro))]

['O3_0.0',
 'RH_0.0',
 'TMP_0.0',
 'hora_9.0',
 'hora_10.0',
 'hora_11.0',
 'hora_12.0',
 'hora_13.0',
 'hora_18.0',
 'hora_19.0',
 'hora_20.0',
 'hora_21.0',
 'hora_22.0',
 'hora_32.0',
 'hora_33.0',
 'hora_34.0',
 'hora_35.0',
 'hora_36.0',
 'hora_42.0',
 'hora_43.0',
 'hora_44.0',
 'hora_45.0',
 'O3_1.0',
 'O3_2.0',
 'O3_3.0',
 'O3_4.0',
 'O3_21.0',
 'O3_22.0',
 'O3_23.0',
 'O3_24.0',
 'O3_25.0',
 'O3_26.0',
 'O3_27.0',
 'O3_45.0',
 'O3_46.0',
 'O3_47.0',
 'O3_48.0',
 'O3_1.0',
 'O3_2.0',
 'O3_3.0',
 'O3_4.0',
 'O3_20.0',
 'O3_21.0',
 'O3_22.0',
 'O3_23.0',
 'O3_24.0',
 'O3_25.0',
 'O3_26.0',
 'O3_27.0',
 'O3_44.0',
 'O3_45.0',
 'O3_46.0',
 'O3_47.0',
 'O3_48.0',
 'CO_5.0',
 'CO_6.0',
 'CO_7.0',
 'CO_8.0',
 'CO_29.0',
 'CO_30.0',
 'CO_31.0',
 'NO_6.0',
 'NO_7.0',
 'NO_8.0',
 'NO_9.0',
 'NO_30.0',
 'NO_31.0',
 'NO_32.0',
 'NOX_6.0',
 'NOX_7.0',
 'NOX_8.0',
 'NOX_9.0',
 'NOX_30.0',
 'NOX_31.0',
 'NOX_32.0',
 'O3_1.0',
 'O3_2.0',
 'O3_3.0',
 'O3_4.0',
 'O3_21.0',
 'O3_22.0',
 'O3_23.0'

In [12]:
estaciones = data_hour_merge_24.id_station.unique().tolist() # recolectamos los nombres de las estaciones
data_est = {}
for elem in estaciones:
    data_est[elem] = data_hour_merge_24[data_hour_merge_24.id_station == elem] #hacemos un df por estación

Aplicamos la función para cada dataframe:

In [62]:
data_shift = {}
for elem in data_est:
    data_shift[elem] = shit_corr(data_est[elem])

Unimos los dataframes:

In [63]:
df_append = pd.DataFrame(columns = data_est["MER"].columns.tolist())
for key in data_est:
    df_append = df_append.append(data_est[key], ignore_index=True,sort=True)

In [64]:
df_append.tail(5)

,CO,NO,NO2,NO2_10.0,NO2_11.0,NO2_12.0,NO2_13.0,NO2_14.0,NO2_15.0,NO2_16.0,...,PM25mean_9.0,RH,SO2,TMP,WSP,dia,fecha,hora,id_station,mes
252933,NaN,NaN,28.0,15.0,14.0,11.0,7.0,6.0,6.0,10.0,...,19.541667,80.0,NaN,14.6,0.9,31.0,2019-07-31 06:00:00,6,GAM,7.0
252934,NaN,NaN,28.0,15.0,15.0,14.0,11.0,7.0,6.0,6.0,...,19.916667,80.0,NaN,14.7,0.0,31.0,2019-07-31 07:00:00,7,GAM,7.0
252935,NaN,NaN,32.0,15.0,15.0,15.0,14.0,11.0,7.0,6.0,...,19.916667,75.0,NaN,15.9,0.8,31.0,2019-07-31 08:00:00,8,GAM,7.0
252936,NaN,NaN,40.0,17.0,15.0,15.0,15.0,14.0,11.0,7.0,...,20.083333,69.0,NaN,17.3,1.3,31.0,2019-07-31 09:00:00,9,GAM,7.0
252937,NaN,NaN,NaN,12.0,17.0,15.0,15.0,15.0,14.0,11.0,...,20.125000,66.0,NaN,17.7,1.4,31.0,2019-07-31 10:00:00,10,GAM,7.0


# 2.
### Se obtiene el promedio, máximo y mínimo por hora de todas las estaciones.

Debido a que solo algunas estaciones miden todos los atributos con los que se está trabajando, se encontró que agrupar las condiciones atmosféricas de cada atributo con el promedio, máximo y mínimo, genera una mejora considerable al desempeño final de los modelos, debido a que al hacer esta agrupación se suavizan los valores y también se homogeniza los atributos de cada estación.

Se guardan las columnas al aplicar el groupby y se elimina "fecha":

In [65]:
cols = df_append.columns.tolist() 
cols.remove("fecha")
cols.remove("id_station")
df_append[cols] = df_append[cols].astype(float)

Aplicamos los groupbys:

In [66]:
data_hour_merge_24_mean = df_append.groupby('fecha')[cols].mean()
data_hour_merge_24_mean.reset_index(inplace=True)

In [67]:
data_hour_merge_24_max = df_append.groupby('fecha')[cols].max()
data_hour_merge_24_max.reset_index(inplace=True)

In [68]:
data_hour_merge_24_min = df_append.groupby('fecha')[cols].min()
data_hour_merge_24_min.reset_index(inplace=True)

#### Cambiamos los nombres de las estaciones:
Debido a que cada groupby genera los mismos nombres, esto puede generar problemas para identificar de qué columna se trata, se cambiarán los nombres a "contaminante_max", "contaminante_min", "contaminante_mean" según corresponda.

Se obtienenn los nonbres de las columnas en forma de lista:

In [69]:
max_columns = data_hour_merge_24_max.columns.tolist()
min_columns = data_hour_merge_24_min.columns.tolist()
mean_columns = data_hour_merge_24_mean.columns.tolist()

Se cambian los nombres en las listas:

In [70]:
for i in range(len(max_columns)):
    if not max_columns[i].startswith("fecha") \
    and not max_columns[i].startswith("hora") \
    and not max_columns[i].startswith("dia") \
    and not max_columns[i].startswith("mes"):
        max_columns[i] = max_columns[i]+"_max"
        
    if not min_columns[i].startswith("fecha") \
    and not min_columns[i].startswith("hora") \
    and not min_columns[i].startswith("dia") \
    and not min_columns[i].startswith("mes"):
        min_columns[i] = min_columns[i]+"_min" 
        
    if not mean_columns[i].startswith("fecha") \
    and not mean_columns[i].startswith("hora") \
    and not mean_columns[i].startswith("dia") \
    and not mean_columns[i].startswith("mes"):
        mean_columns[i] = mean_columns[i]+"_mean"

Se renombra las columnas de los dataframes

In [71]:
data_hour_merge_24_mean.columns = mean_columns
data_hour_merge_24_min.columns = min_columns
data_hour_merge_24_max.columns = max_columns

Unimos los df's 

In [72]:
data_hour_merge = pd.merge(data_hour_merge_24_mean, data_hour_merge_24_max, on=['fecha'])
data_hour_merge = pd.merge(data_hour_merge, data_hour_merge_24_min, on=['fecha'])

In [73]:
data_hour_merge_columns = data_hour_merge.columns.tolist()

###### Eliminamos las columnas dehora dia y mes que acaben con x & y

In [74]:
for item in range(len(data_hour_merge_columns)):
    if  data_hour_merge_columns[item].endswith("_x")\
    or  data_hour_merge_columns[item].endswith("_y"):
        data_hour_merge_columns[item] = data_hour_merge_columns[item][:-2]

In [75]:
data_hour_merge.columns = data_hour_merge_columns#.tolist()

data_hour_merge = data_hour_merge.loc[:,~data_hour_merge.columns.duplicated()]

Debido a que el proceso de obtener el máximo y mínimo es lento, es conveniente guardar el archivo de manera provisional:

In [77]:
data_hour_merge.to_csv("./datos/datos_gb_" + contaminante + ".csv", sep=',', encoding='utf-8',index=False)

# Leamos el archivo:

In [78]:
data_hour_merge = pd.read_csv("./datos/datos_gb_" + contaminante + ".csv", sep=',', encoding='utf-8')

In [79]:
data_hour_merge = data_hour_merge.sort_values(['fecha',"hora"], ascending=[0,1]).reset_index(drop=True)

## 3. 
### Generar las columna a pronosticar y obtener la correlación de los valores promedio máximo y mínimo con éste. Filtrar los valores más importantes.


Ahora vamos a sacar los atributos más correlacionado con el contaminante a pronosticar a pronosticar. 

In [80]:
if contaminante == 'PM10' or contaminante == 'PM2.5':
    item = contaminante+"mean_max"
else:
    item = contaminante+"_max" 
print(item)

PM25mean_max


Desplazamos el target **24 horas** en adelante, esta columna nos indica el valor de contaminación que tomó a partir de condiciones atmosféricas 24 horas atrás. Este será nuesta columna a pronosticar.

In [81]:
data_hour_merge[str(item+"_frcst_"+str(24))] = data_hour_merge[item].shift(24)

In [82]:
target = str(item+"_frcst_"+ str(24))
print(target)

PM25mean_max_frcst_24


In [83]:
data_hour_merge.head()

,fecha,CO_mean,NO_mean,NO2_mean,NO2_10.0_mean,NO2_11.0_mean,NO2_12.0_mean,NO2_13.0_mean,NO2_14.0_mean,NO2_15.0_mean,...,PM25mean_5.0_min,PM25mean_6.0_min,PM25mean_7.0_min,PM25mean_8.0_min,PM25mean_9.0_min,RH_min,SO2_min,TMP_min,WSP_min,PM25mean_max_frcst_24
0,2019-08-01 00:00:00,0.26,1.25,14.6,21.4,27.0,28.6,29.0,26.2,24.8,...,15.750000,16.208333,19.250000,19.458333,18.666667,61.0,1.0,14.8,1.8,NaN
1,2019-07-31 23:00:00,0.40,4.00,20.2,27.0,28.6,29.0,26.2,24.8,25.8,...,16.208333,19.250000,19.458333,18.666667,17.958333,59.0,1.0,15.0,1.3,NaN
2,2019-07-31 22:00:00,0.50,4.00,25.8,28.6,29.0,26.2,24.8,25.8,27.2,...,19.250000,19.458333,18.666667,17.958333,17.291667,56.0,1.0,15.3,1.7,NaN
3,2019-07-31 21:00:00,0.48,3.25,25.0,29.0,26.2,24.8,25.8,27.2,22.6,...,19.458333,18.666667,17.958333,17.291667,16.791667,53.0,1.0,15.0,1.5,NaN
4,2019-07-31 20:00:00,0.44,3.00,20.4,26.2,24.8,25.8,27.2,22.6,15.4,...,18.666667,17.958333,17.291667,16.791667,16.375000,49.0,1.0,14.3,1.6,NaN


## 4. 
### Generar las columnas de pronóstico de RH, WSP TMP.

Se observó que usando datos de pronóstico atmosférico, el modelo tiene un mejor desempeño, vamos a añadir "RH_mean","WSP_mean","TMP_mean" a los atributos con un pronóstico de 24 horas.

In [84]:
lista_frcst = ["RH_mean","WSP_mean","TMP_mean"]

In [85]:
for item_pronostico in lista_frcst:
    for i in range(1, 25):
        col_name = str(item_pronostico+"_frcst_"+str(i))
        data_hour_merge[col_name] = data_hour_merge[item_pronostico].shift(i)
data_hour_merge.dropna(inplace=True)

volvamos a sacar la correlación y filtramos los valores de correlación:

In [86]:
contaminante

'PM25mean'

 Filtramos el número de filas según el contaminante:

In [87]:
if contaminante == "O3":
    threshold = 0.49
elif contaminante == "PM10mean":
        threshold = 0.48
else:
    threshold = 0.44
    

In [88]:
data_corr_2 = data_hour_merge.corr()
variables_mas_correlacionadas_con_pronóstico = data_corr_2[target][(data_corr_2[target] > threshold) |                                                                
                                                            (data_corr_2[target] < - threshold)].index.tolist()

In [89]:
len(variables_mas_correlacionadas_con_pronóstico)

128

Agregamos a las columnas mas correlacionadas los atributos de pronóstico:

In [90]:
variables_mas_correlacionadas_con_pronóstico.append("RH_mean")
variables_mas_correlacionadas_con_pronóstico.append("WSP_mean")
variables_mas_correlacionadas_con_pronóstico.append("TMP_mean")
variables_mas_correlacionadas_con_pronóstico.append("fecha")
variables_mas_correlacionadas_con_pronóstico.append("O3_max")
variables_mas_correlacionadas_con_pronóstico = list(set(variables_mas_correlacionadas_con_pronóstico))

In [91]:
data_hour_merge = data_hour_merge[variables_mas_correlacionadas_con_pronóstico]

In [92]:
variables_mas_correlacionadas_con_pronóstico

['PM25mean_14.0_max',
 'PM10mean_0.0_mean',
 'PM10mean_20.0_mean',
 'PM10_8.0_mean',
 'PM10mean_3.0_max',
 'PM2.5_6.0_max',
 'O3_max',
 'PM10mean_18.0_mean',
 'fecha',
 'PM25mean_2.0_max',
 'PM2.5_9.0_mean',
 'PM10_1.0_mean',
 'PM2.5_2.0_max',
 'PM25mean_0.0_min',
 'PM25mean_13.0_max',
 'PM10mean_10.0_max',
 'PM25mean_5.0_max',
 'PM10mean_15.0_max',
 'PM10mean_6.0_max',
 'PM10mean_0.0_min',
 'NO2_3.0_max',
 'PM10mean_max',
 'PM10_1.0_max',
 'PM10mean_1.0_max',
 'PM10_10.0_mean',
 'PM2.5_4.0_mean',
 'PM10mean_7.0_mean',
 'PM10mean_21.0_mean',
 'PM2.5_mean',
 'PM10mean_23.0_mean',
 'PM25mean_0.0_mean',
 'PM10_4.0_max',
 'PM10mean_17.0_max',
 'PM10_12.0_mean',
 'PM25mean_6.0_max',
 'PM25mean_6.0_mean',
 'PM25mean_1.0_max',
 'PM10mean_6.0_mean',
 'PM10mean_11.0_mean',
 'PM2.5_3.0_max',
 'PM2.5_2.0_mean',
 'PM10mean_22.0_mean',
 'PM10mean_16.0_mean',
 'PM10mean_8.0_mean',
 'PM25mean_11.0_mean',
 'PM25mean_4.0_max',
 'PM25mean_0.0_max',
 'NO2_max',
 'WSP_mean',
 'PM10mean_9.0_mean',
 'PM10me

## 5. 
### Generar las columnas del contaminante a pronosticar de 1 hasta 23 horas.


Al tratarse de modelos supervisados, es necesario tener como columna cada uno de los valores a pronosticar, esta columna es la columna del contaminante recorrido desde 1 hasta 24 horas atrás:

Definimos el contaminante que estamos usando:

In [93]:
if contaminante == 'PM10' or contaminante == 'PM2.5':
    item = contaminante+"mean_max"
else:
    item = contaminante+"_max" 
print(item)

PM25mean_max


In [94]:
for i in range(1, 24):
    col_name = str(item+"_frcst_"+str(i))
    data_hour_merge[col_name] = data_hour_merge[item].shift(i)
    if i==range(1, 24)[-1]:
        print(i)
data_hour_merge.dropna(inplace=True)

23


Guardemos:

In [95]:
data_hour_merge.to_csv("./datos/datos_modelos_"+ contaminante + ".csv",sep=',', encoding='utf-8',index=False)

In [96]:
data_hour_merge.columns.tolist()

['PM25mean_14.0_max',
 'PM10mean_0.0_mean',
 'PM10mean_20.0_mean',
 'PM10_8.0_mean',
 'PM10mean_3.0_max',
 'PM2.5_6.0_max',
 'O3_max',
 'PM10mean_18.0_mean',
 'fecha',
 'PM25mean_2.0_max',
 'PM2.5_9.0_mean',
 'PM10_1.0_mean',
 'PM2.5_2.0_max',
 'PM25mean_0.0_min',
 'PM25mean_13.0_max',
 'PM10mean_10.0_max',
 'PM25mean_5.0_max',
 'PM10mean_15.0_max',
 'PM10mean_6.0_max',
 'PM10mean_0.0_min',
 'NO2_3.0_max',
 'PM10mean_max',
 'PM10_1.0_max',
 'PM10mean_1.0_max',
 'PM10_10.0_mean',
 'PM2.5_4.0_mean',
 'PM10mean_7.0_mean',
 'PM10mean_21.0_mean',
 'PM2.5_mean',
 'PM10mean_23.0_mean',
 'PM25mean_0.0_mean',
 'PM10_4.0_max',
 'PM10mean_17.0_max',
 'PM10_12.0_mean',
 'PM25mean_6.0_max',
 'PM25mean_6.0_mean',
 'PM25mean_1.0_max',
 'PM10mean_6.0_mean',
 'PM10mean_11.0_mean',
 'PM2.5_3.0_max',
 'PM2.5_2.0_mean',
 'PM10mean_22.0_mean',
 'PM10mean_16.0_mean',
 'PM10mean_8.0_mean',
 'PM25mean_11.0_mean',
 'PM25mean_4.0_max',
 'PM25mean_0.0_max',
 'NO2_max',
 'WSP_mean',
 'PM10mean_9.0_mean',
 'PM10me

In [97]:
data_hour_merge['fecha'] =  pd.to_datetime(data_hour_merge['fecha'], format='%Y-%m-%d %H:%M')

import matplotlib.pyplot as plt

fig = plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
ax = fig.gca()

# df_export.(50).plot(x="fecha", y=[ "O3_y"],ax=ax,label=["O3 Medido"],color="dodgerblue")
# df_export.(60).head(100).plot(x="fecha", y=["O3_y_frcst_6"],ax=ax,label=["Pronóstico O3 a 12 horas"],color='darkblue',dashes=[6, 2])

data_hour_merge.head(100).plot(x="fecha", y=["O3_max"],ax=ax,label=["PM10mean ahora"],color="saddlebrown")
data_hour_merge.head(100).plot(x="fecha", y=["O3_max_frcst_22"],ax=ax,label=["PM10 pronostico 12"],color="blue")
data_hour_merge.head(100).plot(x="fecha", y=["O3_21.0_max"],ax=ax,label=["PM10 a pasado"],color='maroon',dashes=[6, 2])


plt.ylabel("PM10[mg/m3]")
plt.title("Comparación del Modelo")


# And a corresponding grid
ax.grid(which='both')

# Or if you want different settings for the grids:
ax.grid(which='minor', alpha=0.01)
ax.grid(which='major', alpha=0.3)



plt.legend()
plt.show()

KeyError: "None of [Index(['O3_max_frcst_22'], dtype='object')] are in the [columns]"